## Let's crawl

In [15]:
%pip install undetected-chromedriver selenium fake_useragent ace_tools

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\phuon\anaconda3\lib\site-packages\torchlight-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


## Brute force time

In [4]:
import itertools
import pandas as pd

# List of tokens
tokens = ["bo", "chim", "cho", "chuot", "ga", "heo", "ho", "meo", "ngua", "tho", "trau", "vit", "voi"]

# Generate all permutations of length 3
combos = [''.join(combo) for combo in itertools.permutations(tokens, 3)]

# Show the total count and a preview
df = pd.DataFrame({
    "captcha_combo": combos
})


# Save all combinations to a text file
file_path = "captcha_combinations.txt"
with open(file_path, "w", encoding="utf-8") as f:
    for combo in combos:
        f.write(combo + "\n")

file_path

'captcha_combinations.txt'

In [ ]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time
import json
import os
import random
import sys
from fake_useragent import UserAgent



def random_sleep(min_seconds, max_seconds):
    time.sleep(random.uniform(min_seconds, max_seconds))

with open("captcha_combinations.txt", encoding="utf-8") as f:
    captcha_combos = [line.strip() for line in f if line.strip()]
random.shuffle(captcha_combos)

def solve_captcha_bruteforce(driver):
    """
    Try each candidate in captcha_combos until the captcha disappears.
    Returns True if solved, False if none worked.
    """

    for candidate in captcha_combos:
        try:
            input_el = driver.find_element(By.ID, "captcha")
            verify_btn = driver.find_element(By.ID, "verify")
            input_el.clear()
            input_el.send_keys(candidate)
            verify_btn.click()

            # Wait a moment for response
            try:
                WebDriverWait(driver, 2).until_not(
                    EC.presence_of_element_located((By.ID, "captcha"))
                )
                print(f"CAPTCHA solved with: {candidate}")
                return True
            except:
                pass  # Still there → try next
        except Exception as e:
            print(f"Error during brute-force attempt: {e}")
            continue

    print("Brute‑force failed: none of the combos worked.")
    return False

# --- Add random user-agent selection ---
ua = UserAgent()
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (X11; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0"
]
user_agent = random.choice(user_agents)

def crawl_alonhadat_selenium(page_num, driver):
    url = f"https://alonhadat.com.vn/nha-dat/can-ban/nha-dat/1/ha-noi/trang--{page_num}.html"
    driver.get(url)
    random_sleep(2, 4)  # Wait for page to load

    for _ in range(random.randint(2, 4)):
        driver.execute_script("window.scrollBy(0, window.innerHeight/2);")
        random_sleep(1, 2)

    items = driver.find_elements(By.CSS_SELECTOR, "div.content-item")
    results = []
    for item in items:
        try:
            title_tag = item.find_element(By.CSS_SELECTOR, "div.ct_title a")
            title = title_tag.text.strip()
            link = "https://alonhadat.com.vn" + title_tag.get_attribute("href")

            date = item.find_element(By.CSS_SELECTOR, "div.ct_date").text.strip()

            try:
                area = item.find_element(By.CSS_SELECTOR, "div.ct_dt").text.strip().replace("Diện tích:", "")
            except NoSuchElementException:
                area = ""

            try:
                price = item.find_element(By.CSS_SELECTOR, "div.ct_price").text.strip().replace("Giá:", "")
            except NoSuchElementException:
                price = ""

            try:
                floors = item.find_element(By.CSS_SELECTOR, "span.floors").get_attribute("title")
            except NoSuchElementException:
                floors = ""

            try:
                bedrooms = item.find_element(By.CSS_SELECTOR, "span.bedroom").get_attribute("title")
            except NoSuchElementException:
                bedrooms = ""

            try:
                address = item.find_element(By.CSS_SELECTOR, "div.ct_dis").text.strip()
            except NoSuchElementException:
                address = ""

            results.append({
                "title": title,
                "url": link,
                "date": date,
                "area": area,
                "price": price,
                "floors": floors,
                "bedrooms": bedrooms,
                "address": address,
            })
        except Exception as e:
            print(f"Error parsing item: {e}")
    return results

def is_captcha_present(driver):
    try:
        # Check for the instruction text or the captcha input box
        body_text = driver.find_element(By.TAG_NAME, "body").text
        if "Nhập tên 3 con vật" in body_text or "Tôi không phải người máy" in body_text:
            return True
        # Or check for the captcha input box by id
        if driver.find_elements(By.ID, "captcha"):
            return True
    except Exception:
        pass
    return False

# --- Setup undetected-chromedriver ---
chrome_options = uc.ChromeOptions()
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument(f"user-agent={user_agent}") 
driver = uc.Chrome(options=chrome_options, version_main=136)

def save_progress(page):
    with open("selenium_progress.txt", "w") as f:
        f.write(str(page))

def load_progress():
    if os.path.exists("selenium_progress.txt"):
        with open("selenium_progress.txt", "r") as f:
            return int(f.read().strip())
    return None


# --- Crawl pages and save as JSONL ---
output_folder = "../Datasets/alonhadat.com/json_new"
os.makedirs(output_folder, exist_ok=True)
start_page = load_progress() + 1 if load_progress() is not None else 1
end_page = 2000

for page_num in range(start_page, end_page + 1):
    print(f"Crawling page {page_num}...")

    # CAPTCHA check and manual intervention
    while is_captcha_present(driver):
        # print("\n[CAPTCHA detected!]")
        # solved = solve_captcha_bruteforce(driver)
        # if not solved:
        #     print("Please solve CAPTCHA manually, then press Enter to continue...")
        #     input()
        # else:
        #     # Let the page settle after solving
        #     time.sleep(1)

        # # Manual    
        # print("Please solve the CAPTCHA in the browser window, then press Enter here to continue...")
        # input()
        # driver.refresh()
        # random_sleep(2, 4)

        # Super manual
        sys.stdout.write(".")
        sys.stdout.flush()
        random_sleep(1, 3)
    
    data = crawl_alonhadat_selenium(page_num, driver)
    if data:
        output_file = os.path.join(output_folder, f"page_{page_num}.jsonl")
        with open(output_file, "w", encoding="utf-8") as file:
            for d in data:
                file.write(json.dumps(d, ensure_ascii=False) + "\n")
        print(f"Saved {len(data)} items to {output_file}")
    else:
        print(f"No data found for page {page_num}")
    save_progress(page_num)
    random_sleep(1, 3)  # Be polite to the server

driver.quit()

Crawling page 1670...
Saved 20 items to ../Datasets/alonhadat.com/json_new\page_1670.jsonl
Crawling page 1671...
Saved 20 items to ../Datasets/alonhadat.com/json_new\page_1671.jsonl
Crawling page 1672...
Saved 20 items to ../Datasets/alonhadat.com/json_new\page_1672.jsonl
Crawling page 1673...
Saved 20 items to ../Datasets/alonhadat.com/json_new\page_1673.jsonl
Crawling page 1674...
Saved 20 items to ../Datasets/alonhadat.com/json_new\page_1674.jsonl
Crawling page 1675...
Saved 20 items to ../Datasets/alonhadat.com/json_new\page_1675.jsonl
Crawling page 1676...
Saved 20 items to ../Datasets/alonhadat.com/json_new\page_1676.jsonl
Crawling page 1677...
Saved 20 items to ../Datasets/alonhadat.com/json_new\page_1677.jsonl
Crawling page 1678...
Saved 20 items to ../Datasets/alonhadat.com/json_new\page_1678.jsonl
Crawling page 1679...
Saved 20 items to ../Datasets/alonhadat.com/json_new\page_1679.jsonl
Crawling page 1680...
Saved 20 items to ../Datasets/alonhadat.com/json_new\page_1680.jsonl